In [37]:
# load necessary python libraries

import pandas as pd

In [42]:
# read the original cvs file into Pandas for the Kepler Object data

x_orig_data = pd.read_csv('x_parms.csv')
x_orig_data.head()


,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,0,0,0,0,9.488036,2.775000e-05,-2.775000e-05,170.538750,0.002160,-0.002160,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,-0.003520,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,0,1,0,0,19.899140,1.494000e-05,-1.494000e-05,175.850252,0.000581,-0.000581,...,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,-0.000115,...,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,0,0,0,0,2.525592,3.761000e-06,-3.761000e-06,171.595550,0.001130,-0.001130,...,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509


In [43]:
# read the reduced by decision tree, X parameter cvs file into Pandas for the Kepler Object data

x_reduced_data = pd.read_csv('x_filtered_dt.csv')
x_reduced_data.head()


,koi_fpflag_nt,koi_fpflag_co,koi_fpflag_ss,koi_model_snr,koi_fpflag_ec,koi_impact
0,0,0,0,35.8,0,0.146
1,0,0,0,25.8,0,0.586
2,0,0,1,76.3,0,0.969
3,0,0,1,505.6,0,1.276
4,0,0,0,40.9,0,0.701


In [44]:
# read y values into Pandas for Kepler classification data

y_data = pd.read_csv('y_value.csv')
y_data.head()

,koi_disposition
0,CONFIRMED
1,CONFIRMED
2,FALSE POSITIVE
3,FALSE POSITIVE
4,CONFIRMED


In [45]:
# load Random Forest classifier library
from sklearn.ensemble import RandomForestClassifier

In [46]:
# Create a random forest classifier for original data
rf = RandomForestClassifier(n_estimators=200)


In [48]:
# define training and test data for the Random Forest model
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_orig_data, y_data, random_state=5)

In [49]:
# fit model with original data
rf = rf.fit(X_train, y_train)

C:\Users\wavau\anaconda3\envs\pythondata\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [51]:
#score model with original full data
rf.score(X_test, y_test)

0.9007319304666057

## Random Forest Classification Score: 0.9007
### Better than DCF Model: 0.8417

In [52]:
# Random Forests calculate feature importance
importances = rf.feature_importances_
importances

array([0.10254355, 0.07324628, 0.10979645, 0.03651035, 0.02188827,
       0.01687229, 0.01488162, 0.01399507, 0.02346086, 0.02153424,
       0.01910797, 0.01123521, 0.01109209, 0.02372327, 0.03175829,
       0.03505492, 0.02105807, 0.01253968, 0.0124264 , 0.04996391,
       0.03255438, 0.02982602, 0.01513277, 0.01396382, 0.01641693,
       0.01618936, 0.05385536, 0.00342238, 0.00959927, 0.02946266,
       0.027344  , 0.00892357, 0.00843618, 0.01016335, 0.00886868,
       0.01241292, 0.00816479, 0.01241448, 0.01034792, 0.00981237])

In [53]:
# evaluate feature importance from X parameters
features_df = pd.DataFrame(zip(x_orig_data.columns, rf.feature_importances_))
# sort by feature importance 
features_df = features_df.sort_values(by=[1], ascending=False)
# features_df

#feature importance is much longer with random forest model vs. decision tree model
#16 parameter with more than 0.02 feature importance vs. 6 with decision tree model
# NOTE all parameters from the Decision Tree model exceed the 0.02 feature importance
# with the RF model as well.  

In [54]:
# drop all variables less than 0.02 feature importance
x_filtered_df = features_df[features_df[1]>=0.02]
x_filtered_df

,0,1
2,koi_fpflag_co,0.109796
0,koi_fpflag_nt,0.102544
1,koi_fpflag_ss,0.073246
26,koi_model_snr,0.053855
19,koi_prad,0.049964
3,koi_fpflag_ec,0.036510
15,koi_duration_err2,0.035055
20,koi_prad_err1,0.032554
14,koi_duration_err1,0.031758
21,koi_prad_err2,0.029826


In [55]:
df_list1=[]
df_list1 = x_filtered_df[0].values.tolist()
df_list1

['koi_fpflag_co',
 'koi_fpflag_nt',
 'koi_fpflag_ss',
 'koi_model_snr',
 'koi_prad',
 'koi_fpflag_ec',
 'koi_duration_err2',
 'koi_prad_err1',
 'koi_duration_err1',
 'koi_prad_err2',
 'koi_steff_err1',
 'koi_steff_err2',
 'koi_duration',
 'koi_time0bk_err1',
 'koi_period',
 'koi_time0bk_err2',
 'koi_depth']

In [56]:
# create new dataframe from reduced x_parameters decision tree output
x_r_forsest_filt_df = x_orig_data[df_list1]
x_r_forsest_filt_df.head()

,koi_fpflag_co,koi_fpflag_nt,koi_fpflag_ss,koi_model_snr,koi_prad,koi_fpflag_ec,koi_duration_err2,koi_prad_err1,koi_duration_err1,koi_prad_err2,koi_steff_err1,koi_steff_err2,koi_duration,koi_time0bk_err1,koi_period,koi_time0bk_err2,koi_depth
0,0,0,0,35.8,2.26,0,-0.08190,0.26,0.08190,-0.15,81.0,-81.0,2.95750,0.002160,9.488036,-0.002160,615.8
1,0,0,0,25.8,2.83,0,-0.11600,0.32,0.11600,-0.19,81.0,-81.0,4.50700,0.003520,54.418383,-0.003520,874.8
2,0,0,1,76.3,14.60,0,-0.03410,3.92,0.03410,-1.31,158.0,-176.0,1.78220,0.000581,19.899140,-0.000581,10829.0
3,0,0,1,505.6,33.46,0,-0.00537,8.50,0.00537,-2.83,157.0,-174.0,2.40641,0.000115,1.736952,-0.000115,8079.2
4,0,0,0,40.9,2.75,0,-0.04200,0.88,0.04200,-0.35,169.0,-211.0,1.65450,0.001130,2.525592,-0.001130,603.3


In [57]:
# write out new x parameters data file BASED ON RANDOM FOREST FEATURE IMPORTANCE
x_filtered_df.to_csv("x_rand_for_filt.csv", index=False, header=True)